In [2]:
!pip install sqlalchemy==1.4.46

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 1.6/1.6 MB 1.6 MB/s eta 0:00:00
     -------------------------------------- 192.2/192.2 kB 2.3 MB/s eta 0:00:00


In [3]:
import os

import pandas as pd
from sqlalchemy import create_engine

import datetime

In [158]:
posgres_username = "user"
posgres_password = "password"
hostname = "79.137.206.67:5432"
database_name = "traveldata"

engine_setup = 'postgresql+psycopg2://{}:{}@{}/{}'.format(
    posgres_username,
    posgres_password,
    hostname,
    database_name
)

In [159]:
# create engine 
engine = create_engine(engine_setup)

In [12]:
data_root = "./data/"

### download_data

In [13]:
data = pd.read_json(
    os.path.join(
        data_root,
        "TA_places_with_YANDEX_addresses.json"
    ), 
    orient='split',
    encoding="utf-8"
)

In [14]:
data.head()

,place_name,place_tags,place_address,place_rating,reviews_qty,reviews,place_url,longitude,latitude,yandex_place_address,postal_code,house,street,locality,province,country
0,Набережная Pеки Тура,[Пирсы и пляжные променады],"ул. Республики, 4, Тюмень 625003 Россия",4.5,639,"[{'review_title': 'Интересно прогуляться', 're...",https://www.tripadvisor.ru/Attraction_Review-g...,65.522371,57.161884,"Россия, Тюмень, улица Республики, 4",NaN,4,улица Республики,Тюмень,Тюменская область,Россия
1,Мост Влюбленных (Пешеходный),[Мосты],"ул.Республики и ул.Ленина, Тюмень 625026 Россия",4.5,516,"[{'review_title': 'Мост влюбленных. Красота', ...",https://www.tripadvisor.ru/Attraction_Review-g...,65.533429,57.154605,"Россия, Тюмень, улица Ленина",NaN,None,улица Ленина,Тюмень,Тюменская область,Россия
2,Сквер сибирских кошек,[Парки],"ул.Первомайская, 11, Тюмень 625000 Россия",4.0,298,"[{'review_title': 'Забавно', 'review_body': 'К...",https://www.tripadvisor.ru/Attraction_Review-g...,65.536708,57.153741,"Россия, Тюмень, Первомайская улица, 11",625000.0,11,Первомайская улица,Тюмень,Тюменская область,Россия
3,Свято-троицкий мужской монастырь,[Церкви и соборы],"Коммунистическая, 10, Тюмень 625001 Россия",5.0,58,"[{'review_title': 'БЛАГОДАТНОЕ МЕСТО', 'review...",https://www.tripadvisor.ru/Attraction_Review-g...,65.512840,57.169863,"Россия, Тюмень, Коммунистическая улица, 10",625001.0,10,Коммунистическая улица,Тюмень,Тюменская область,Россия
4,Памятник бездомной собаке,"[Культурные объекты и достопримечательности, П...","ул. Орджоникидзе, 63, Тюмень 625002 Россия",4.5,93,"[{'review_title': 'Памятник-копилка', 'review_...",https://www.tripadvisor.ru/Attraction_Review-g...,65.541999,57.149357,"Россия, Тюмень, улица Орджоникидзе, 63",625000.0,63,улица Орджоникидзе,Тюмень,Тюменская область,Россия


-----

In [50]:
places = data[["place_name"]]

In [55]:
places.to_sql(
    name="places", 
    con=engine, 
    schema="dbo",
    index=False,
    method="multi",
    if_exists="append" # append - добавляет строки в существующую таблицу /
)                      # replace - дропает таблицу и записывает заново

199

----

In [134]:
tags = data[["place_tags"]]

In [135]:
tags.insert(0, "place_id", data.index+1)

In [136]:
tags.columns = ["place_id", "tag"]

In [138]:
tags = tags.explode("tag").copy()

In [139]:
tags.head(10)

,place_id,tag
0,1,Пирсы и пляжные променады
1,2,Мосты
2,3,Парки
3,4,Церкви и соборы
4,5,Культурные объекты и достопримечательности
4,5,Памятники и статуи
5,6,Церкви и соборы
6,7,Театры
7,8,Парки
8,9,Культурные объекты и достопримечательности


In [140]:
tags.to_sql(
    name="tags", 
    con=engine, 
    schema="dbo",
    index=False,
    method="multi",
    if_exists="replace" # append - добавляет строки в существующую таблицу /
)                      # replace - дропает таблицу и записывает заново

227

----

In [184]:
comments = data[["reviews"]]

In [185]:
comments.insert(0, "place_id", data.index+1)

In [186]:
comments = comments.explode("reviews").copy()

In [187]:
comments.columns = ["place_id", "comment"]

In [188]:
comments.to_sql(
    name="comments", 
    con=engine, 
    schema="dbo",
    index=False,
    method="multi",
    dtype={"comment": sqlalchemy.types.JSON},
    if_exists="replace" # append - добавляет строки в существующую таблицу /
)                      # replace - дропает таблицу и записывает заново

3841

----

In [172]:
place_info = data[["place_rating",	"reviews_qty"]]

In [173]:
place_info.insert(0, "place_id", data.index+1)

In [174]:
place_info.insert(1, "info_datetime", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

In [175]:
place_info.head()

,place_id,info_datetime,place_rating,reviews_qty
0,1,2023-03-08 16:23:37,4.5,639
1,2,2023-03-08 16:23:37,4.5,516
2,3,2023-03-08 16:23:37,4.0,298
3,4,2023-03-08 16:23:37,5.0,58
4,5,2023-03-08 16:23:37,4.5,93


In [181]:
place_info.to_sql(
    name="place_info", 
    con=engine, 
    schema="dbo",
    index=False,
    method="multi",
    if_exists="replace" # append - добавляет строки в существующую таблицу /
)                      # replace - дропает таблицу и записывает заново

199

---

Дима, прогони у себя код ниже. Похоже у меня нет прав на запись в бд

ProgrammingError: (psycopg2.errors.InsufficientPrivilege) permission denied for relation coordinates

Вроде не критично, но:
- в таблице coordinates в колоке longitude пропущена буква i
- в имени таблицы addresses пропущена буква d

In [39]:
DB_URL = os.getenv("DATABASE_URL")
engine = create_engine(DB_URL)

In [40]:
coordinates = data[["longitude", "latitude"]].copy()

#  в имени колонки longitude пропущена буква
coordinates.rename({'longitude': 'longtude'} , axis = 1, inplace=True)

In [41]:
coordinates.insert(0, "place_id", data.index+1)

In [42]:
coordinates.to_sql(
    name="coordinates", 
    con=engine, 
    schema="dbo",
    index=False,
    method="multi",
    if_exists="append" # append - добавляет строки в существующую таблицу /
)                      # replace - дропает таблицу и записывает заново

ProgrammingError: (psycopg2.errors.InsufficientPrivilege) permission denied for relation coordinates

[SQL: INSERT INTO dbo.coordinates (place_id, longtude, latitude) VALUES (%(place_id_m0)s, %(longtude_m0)s, %(latitude_m0)s), (%(place_id_m1)s, %(longtude_m1)s, %(latitude_m1)s), (%(place_id_m2)s, %(longtude_m2)s, %(latitude_m2)s), (%(place_id_m3)s, %(longtude_m3)s, %(latitude_m3)s), (%(place_id_m4)s, %(longtude_m4)s, %(latitude_m4)s), (%(place_id_m5)s, %(longtude_m5)s, %(latitude_m5)s), (%(place_id_m6)s, %(longtude_m6)s, %(latitude_m6)s), (%(place_id_m7)s, %(longtude_m7)s, %(latitude_m7)s), (%(place_id_m8)s, %(longtude_m8)s, %(latitude_m8)s), (%(place_id_m9)s, %(longtude_m9)s, %(latitude_m9)s), (%(place_id_m10)s, %(longtude_m10)s, %(latitude_m10)s), (%(place_id_m11)s, %(longtude_m11)s, %(latitude_m11)s), (%(place_id_m12)s, %(longtude_m12)s, %(latitude_m12)s), (%(place_id_m13)s, %(longtude_m13)s, %(latitude_m13)s), (%(place_id_m14)s, %(longtude_m14)s, %(latitude_m14)s), (%(place_id_m15)

----

Тут у меня получилось создать новую таблицу с именем "addresses".

Но не получается записать в уже существующую "adresses"

In [47]:
addresses = data[["postal_code", "house", "street", "locality", "province", "country"]]

In [44]:
addresses.insert(0, "place_id", data.index+1)

In [49]:
addresses.to_sql(
    name="addresses", 
    con=engine, 
    schema="dbo",
    index=False,
    method="multi",
    if_exists="replace" # append - добавляет строки в существующую таблицу /
)                      # replace - дропает таблицу и записывает заново

199